In [120]:
import pandas as pd
import os
import numpy as np
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Download dataset from "https://www.kaggle.com/datasets/nasrulhakim86/coughvid-wav"
# Extract it and add the location paths

In [121]:
audio_dataset_path = 'D:/work3/Omdena/NewDelhi_Chapter/data/archive/public_dataset/'
metadata_file_path = 'D:/work3/Omdena/NewDelhi_Chapter/data/archive/public_dataset/metadata_compiled.csv'

In [122]:
metadf = pd.read_csv(metadata_file_path)
metadf.head()

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,...,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
0,00014dcc-0f06-4c27-8c7b-737b18a2cf4c,2020-11-25T18:58:50.488301+00:00,0.0155,7.326171,48.9,2.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,16.151433,31.3,34.8,15.0,male,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,2020-10-18T15:38:38.205870+00:00,0.1643,16.217201,NaN,NaN,46.0,female,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,20.146058,40.0,-75.1,34.0,male,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,0.000000,-16.5,-71.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
print(metadf.columns)

Index(['uuid', 'datetime', 'cough_detected', 'SNR', 'latitude', 'longitude',
       'age', 'gender', 'respiratory_condition', 'fever_muscle_pain', 'status',
       'quality_1', 'cough_type_1', 'dyspnea_1', 'wheezing_1', 'stridor_1',
       'choking_1', 'congestion_1', 'nothing_1', 'diagnosis_1', 'severity_1',
       'quality_2', 'cough_type_2', 'dyspnea_2', 'wheezing_2', 'stridor_2',
       'choking_2', 'congestion_2', 'nothing_2', 'diagnosis_2', 'severity_2',
       'quality_3', 'cough_type_3', 'dyspnea_3', 'wheezing_3', 'stridor_3',
       'choking_3', 'congestion_3', 'nothing_3', 'diagnosis_3', 'severity_3',
       'quality_4', 'cough_type_4', 'dyspnea_4', 'wheezing_4', 'stridor_4',
       'choking_4', 'congestion_4', 'nothing_4', 'diagnosis_4', 'severity_4'],
      dtype='object')


In [50]:
metadf['status'].value_counts()

healthy        12479
symptomatic     2590
COVID-19        1155
Name: status, dtype: int64

In [310]:
#metadf['respiratory_condition'].value_counts()

False    13403
True      2821
Name: respiratory_condition, dtype: int64

In [311]:
#metadf['fever_muscle_pain'].value_counts()

False    14296
True      1928
Name: fever_muscle_pain, dtype: int64

In [312]:
#metadf['cough_detected'].value_counts()

1.0000    2816
0.0000     106
0.9956      55
0.9958      52
0.9955      51
          ... 
0.1348       1
0.0027       1
0.1947       1
0.4124       1
0.5257       1
Name: cough_detected, Length: 7212, dtype: int64

In [51]:
metadf2 = metadf[((metadf['cough_detected'] > 0.99) & (metadf['status'] == 'healthy')) |
                 ((metadf['cough_detected'] > 0) & (metadf['status'] == 'COVID-19')) |
                 ((metadf['cough_detected'] > 0) & (metadf['status'] == 'symptomatic'))]

In [369]:
# This would create a bit more balanced dataset
'''
metadf2 = metadf[((metadf['cough_detected'] > 0.996) & (metadf['status'] == 'healthy')) |
                 ((metadf['cough_detected'] > 0.5) & (metadf['status'] == 'COVID-19')) |
                 ((metadf['cough_detected'] > 0.98) & (metadf['status'] == 'symptomatic'))]
'''

In [52]:
metadf2['status'].value_counts()

healthy        2761
symptomatic    2575
COVID-19       1133
Name: status, dtype: int64

In [389]:
#metadf2['cough_detected'].value_counts()

1.0000    2500
0.9958      42
0.9955      42
0.9936      39
0.9956      38
          ... 
0.8955       1
0.0260       1
0.5336       1
0.3786       1
0.5257       1
Name: cough_detected, Length: 1581, dtype: int64

In [316]:
#metadf2['respiratory_condition'].value_counts()

False    2878
True      982
Name: respiratory_condition, dtype: int64

In [317]:
#metadf2['fever_muscle_pain'].value_counts()

False    3095
True      765
Name: fever_muscle_pain, dtype: int64

In [319]:
#metadf2['quality_1'].value_counts()

ok      521
poor    137
good     31
Name: quality_1, dtype: int64

In [320]:
#metadf2['cough_type_1'].value_counts()

dry        368
unknown    263
wet         58
Name: cough_type_1, dtype: int64

In [321]:
#metadf2['dyspnea_1'].value_counts()

False    685
True       4
Name: dyspnea_1, dtype: int64

In [322]:
#metadf2['wheezing_1'].value_counts()

False    640
True      49
Name: wheezing_1, dtype: int64

In [323]:
#metadf2['stridor_1'].value_counts()

False    689
Name: stridor_1, dtype: int64

In [324]:
#metadf2['choking_1'].value_counts()

False    688
True       1
Name: choking_1, dtype: int64

In [325]:
#metadf2['congestion_1'].value_counts()

False    675
True      14
Name: congestion_1, dtype: int64

In [326]:
#metadf2['nothing_1'].value_counts()

True     625
False     64
Name: nothing_1, dtype: int64

In [327]:
#metadf2['diagnosis_1'].value_counts()

COVID-19               244
healthy_cough          207
lower_infection        204
upper_infection         20
obstructive_disease     14
Name: diagnosis_1, dtype: int64

In [328]:
#metadf2['severity_1'].value_counts()

mild           447
pseudocough    199
severe          31
unknown         12
Name: severity_1, dtype: int64

In [53]:
newdf = metadf2

In [272]:
'''
newdf = metadf[(((metadf['cough_detected'] > 0.5) & (metadf['status'] == 'healthy')) 
                & (((metadf['quality_1'] == 'good') | (metadf['quality_1'] == 'ok')) |
                 ((metadf['quality_2'] == 'good') | (metadf['quality_2'] == 'ok')) |
                 ((metadf['quality_3'] == 'good') | (metadf['quality_3'] == 'ok')) |
                 ((metadf['quality_4'] == 'good') | (metadf['quality_4'] == 'ok')))) 
                 |
                 (((metadf['cough_detected'] > 0.5) & (metadf['status'] == 'symptomatic')) 
                & (((metadf['quality_1'] == 'good') | (metadf['quality_1'] == 'ok')) |
                 ((metadf['quality_2'] == 'good') | (metadf['quality_2'] == 'ok')) |
                 ((metadf['quality_3'] == 'good') | (metadf['quality_3'] == 'ok')) |
                 ((metadf['quality_4'] == 'good') | (metadf['quality_4'] == 'ok'))))
                 |
                 ((metadf['cough_detected'] > 0) & (metadf['status'] == 'COVID-19'))]
'''

In [54]:
newdf

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,...,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
5,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,13.146502,NaN,NaN,21.0,male,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,001d8e33-a4af-4edb-98ba-b03f891d9a6c,2020-05-13T01:27:42.552773+00:00,0.0306,12.713480,13.8,-89.6,NaN,female,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0029d048-898a-4c70-89c7-0815cdcf7391,2020-07-16T03:01:55.059762+00:00,1.0000,9.624196,NaN,NaN,35.0,male,True,False,...,good,dry,False,False,False,False,False,True,upper_infection,mild
13,002d28bc-7806-4dfb-9c9b-afa8cb623cac,2020-04-16T22:10:24.107938+00:00,0.9959,35.641847,NaN,NaN,46.0,female,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,00357712-dd5a-4c0a-90a4-39f1f4b9d5fd,2020-04-10T12:48:17.554497+00:00,0.0576,0.000000,46.8,6.6,NaN,male,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27534,ffd18a56-096d-40fc-9862-e5c5a8ca1fcd,2020-04-14T05:56:02.587585+00:00,1.0000,12.290065,46.5,6.6,25.0,female,False,False,...,ok,dry,False,False,False,False,False,True,COVID-19,mild
27539,ffe0658f-bade-4654-ad79-40a468aabb03,2020-04-14T01:58:32.200245+00:00,1.0000,21.960583,41.6,60.9,22.0,male,True,True,...,poor,unknown,False,False,False,False,False,False,NaN,unknown
27540,ffe13fcf-c5c2-4a6a-a9fc-e010f4f033c1,2020-04-13T21:08:50.708320+00:00,0.9485,9.966762,41.1,28.8,31.0,male,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27542,ffedc843-bfc2-4ad6-a749-2bc86bdac84a,2020-06-05T03:41:37.481463+00:00,1.0000,33.661082,-34.5,-58.5,23.0,male,False,False,...,good,dry,False,False,False,False,False,True,healthy_cough,pseudocough


In [55]:
# This indicates we have a somewhat balanced dataset
newdf['status'].value_counts()

healthy        2761
symptomatic    2575
COVID-19       1133
Name: status, dtype: int64

In [56]:
newdf = newdf.fillna("")

In [57]:
newdf.drop(newdf.loc[newdf['status']=="unknown"].index, inplace=True)
newdf.drop(newdf.loc[newdf['status']==""].index, inplace=True)


In [58]:
newdf

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,...,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
5,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,13.146502,,,21.0,male,False,False,...,,,,,,,,,,
7,001d8e33-a4af-4edb-98ba-b03f891d9a6c,2020-05-13T01:27:42.552773+00:00,0.0306,12.713480,13.8,-89.6,,female,False,True,...,,,,,,,,,,
12,0029d048-898a-4c70-89c7-0815cdcf7391,2020-07-16T03:01:55.059762+00:00,1.0000,9.624196,,,35.0,male,True,False,...,good,dry,False,False,False,False,False,True,upper_infection,mild
13,002d28bc-7806-4dfb-9c9b-afa8cb623cac,2020-04-16T22:10:24.107938+00:00,0.9959,35.641847,,,46.0,female,False,False,...,,,,,,,,,,
17,00357712-dd5a-4c0a-90a4-39f1f4b9d5fd,2020-04-10T12:48:17.554497+00:00,0.0576,0.000000,46.8,6.6,,male,True,False,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27534,ffd18a56-096d-40fc-9862-e5c5a8ca1fcd,2020-04-14T05:56:02.587585+00:00,1.0000,12.290065,46.5,6.6,25.0,female,False,False,...,ok,dry,False,False,False,False,False,True,COVID-19,mild
27539,ffe0658f-bade-4654-ad79-40a468aabb03,2020-04-14T01:58:32.200245+00:00,1.0000,21.960583,41.6,60.9,22.0,male,True,True,...,poor,unknown,False,False,False,False,False,False,,unknown
27540,ffe13fcf-c5c2-4a6a-a9fc-e010f4f033c1,2020-04-13T21:08:50.708320+00:00,0.9485,9.966762,41.1,28.8,31.0,male,False,False,...,,,,,,,,,,
27542,ffedc843-bfc2-4ad6-a749-2bc86bdac84a,2020-06-05T03:41:37.481463+00:00,1.0000,33.661082,-34.5,-58.5,23.0,male,False,False,...,good,dry,False,False,False,False,False,True,healthy_cough,pseudocough


In [63]:
status_label_map = {"healthy":0, "symptomatic": 1, "COVID-19": 2}
newdf = newdf.replace({"status": status_label_map})

In [64]:
newdf.status.value_counts()

0    2761
1    2575
2    1133
Name: status, dtype: int64

In [65]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name)

    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=2048, hop_length=512, n_mels=20)
    mel_scaled_features = np.mean(mel_spectrogram.T,axis=0)

    #stft = np.abs(librosa.stft(audio))
    #chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate)
    #chroma_scaled_features = np.mean(chroma.T,axis=0)


    return mfccs_scaled_features, mel_scaled_features #, chroma_scaled_features
    

In [66]:
import numpy as np
from tqdm import tqdm
# Extract Mel Frequency Cepstral Coefficients and Mel Spectogram
extracted_features=[]
for index_num,row in tqdm(newdf.iterrows()):
    file_name = os.path.join(audio_dataset_path,str(row["uuid"])+".wav")
    final_class_labels=row["status"]
    mfcc, melspec = features_extractor(file_name)
    extracted_features.append([mfcc,melspec,final_class_labels])

6469it [06:17, 17.15it/s]


In [67]:
### converting extracted_features to Pandas dataframe
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature1','feature2','status'])
extracted_features_df.head()

,feature1,feature2,status
0,"[-519.8622, 26.079721, -28.713432, 7.9876924, ...","[0.7552014, 1.6019976, 1.7519484, 0.89610916, ...",0
1,"[-517.534, 125.805275, -59.390606, -13.782511,...","[0.05159273, 0.09458568, 0.079764344, 0.082790...",2
2,"[-492.71045, 16.865747, -3.735841, -0.9161602,...","[4.0721083, 3.1780303, 1.1660261, 0.21596567, ...",1
3,"[-507.23135, 19.040976, -16.899456, 5.4804497,...","[1.23425, 1.3887762, 3.1613564, 5.6069775, 0.7...",0
4,"[-957.87195, -0.9061077, -0.47938585, -3.27202...","[1.8812892e-07, 4.6140246e-07, 2.4876701e-06, ...",2


In [68]:
extracted_features_df.shape

(6469, 3)

In [18]:
#extracted_features_df.to_parquet("mfcc120_melspec120_coughvid.parquet", index=False)

In [69]:
### Split the dataset into independent and dependent dataset
X1 = np.array(extracted_features_df['feature1'].tolist())
X2 = np.array(extracted_features_df['feature2'].tolist())
#X3 = np.array(extracted_features_df['feature3'].tolist())
X = np.concatenate((X1, X2), axis=1)
#X = np.concatenate((X, X3), axis=1)
y = np.array(extracted_features_df['status'].tolist())

In [70]:
X

array([[-5.19862183e+02,  2.60797215e+01, -2.87134323e+01, ...,
         6.74860850e-02,  6.51411572e-03,  2.23455629e-07],
       [-5.17533997e+02,  1.25805275e+02, -5.93906059e+01, ...,
         1.39014261e-07,  2.55018993e-08,  1.24391333e-08],
       [-4.92710449e+02,  1.68657475e+01, -3.73584104e+00, ...,
         1.42640760e-02,  2.26114839e-02,  3.19071822e-02],
       ...,
       [-4.77452209e+02,  5.04670677e+01, -2.13637428e+01, ...,
         7.58189857e-02,  4.89415554e-03,  3.58007753e-07],
       [-4.20832428e+02,  2.41599216e+01, -7.46264648e+00, ...,
         5.03229257e-03,  4.55800124e-04,  3.49893909e-08],
       [-4.48928497e+02,  3.12053547e+01, -5.50211754e+01, ...,
         1.14069000e-04,  3.62838750e-06,  4.13157997e-09]], dtype=float32)

In [71]:
X.shape

(6469, 40)

In [94]:
### Label Encoding
y = np.array(pd.get_dummies(y))

In [95]:
y

array([[1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0]], dtype=uint8)

In [96]:
y.shape

(6469, 3)

In [97]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [98]:
from sklearn.model_selection import train_test_split
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,test_size=0.5,random_state=0)

In [99]:
X_train

array([[-4.5111914e+02,  4.7152843e+01, -7.9044811e-02, ...,
         4.1990317e-02,  1.2785282e-03,  1.2576055e-07],
       [-4.7834470e+02,  9.5170235e+01, -8.1451263e+00, ...,
         1.1002996e-03,  5.2689970e-04,  1.8547170e-04],
       [-3.7469168e+02,  4.0205624e+01, -1.5869018e+01, ...,
         2.3886393e-01,  7.9907961e-03,  3.1211734e-05],
       ...,
       [-5.2719574e+02,  2.8273001e+01, -1.0320598e+01, ...,
         1.0576471e-03,  1.2776282e-04,  1.0527529e-08],
       [-4.9230304e+02,  5.6945549e+01, -4.4023633e+00, ...,
         9.3883275e-05,  1.2435144e-06,  4.3175548e-09],
       [-3.7632587e+02,  4.5872734e+01, -1.6520857e+01, ...,
         6.4031027e-02,  4.6478701e-03,  9.4735225e-07]], dtype=float32)

In [100]:
y

array([[1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0]], dtype=uint8)

In [101]:
X_train.shape

(4528, 40)

In [102]:
X_val.shape

(970, 40)

In [103]:
X_test.shape

(971, 40)

In [104]:
y_train.shape

(4528, 3)

In [105]:
y_val.shape

(970, 3)

In [106]:
y_test.shape

(971, 3)

In [ ]:
# TRAIN an XGBOOST MODEL

In [107]:
# Define the model
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()

In [108]:
# Train the model
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [109]:
# Get the predictions
y_pred = model.predict(X_test)
predictions = [np.round(x) for x in y_pred]

In [110]:
# Check the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 42.95%


In [ ]:
# Train an ANN model

In [111]:
# Define the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

### No of classes
num_labels = y.shape[1]

model=Sequential()
#First Layer
model.add(Dense(1024,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.50))

#Second Layer
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.30))

#Third Layer
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Fourth Layer
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Final Layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [112]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 1024)              41984     
                                                                 
 activation_10 (Activation)  (None, 1024)              0         
                                                                 
 dropout_8 (Dropout)         (None, 1024)              0         
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
 activation_11 (Activation)  (None, 512)               0         
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 256)              

In [113]:
# Compile the model with loss and optimizer
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [114]:
# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='audio_classification_v2.0.hdf5', verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_val, y_val), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/200
142/142 [==============================] - ETA: 0s - loss: 4.1936 - accuracy: 0.3801
Epoch 1: val_loss improved from inf to 1.07835, saving model to audio_classification_v2.0.hdf5
142/142 [==============================] - 3s 13ms/step - loss: 4.1936 - accuracy: 0.3801 - val_loss: 1.0783 - val_accuracy: 0.3639
Epoch 2/200
141/142 [============================>.] - ETA: 0s - loss: 1.2547 - accuracy: 0.4014
Epoch 2: val_loss did not improve from 1.07835
142/142 [==============================] - 2s 11ms/step - loss: 1.2543 - accuracy: 0.4013 - val_loss: 1.0991 - val_accuracy: 0.4742
Epoch 3/200
140/142 [============================>.] - ETA: 0s - loss: 1.1310 - accuracy: 0.4062
Epoch 3: val_loss improved from 1.07835 to 1.04778, saving model to audio_classification_v2.0.hdf5
142/142 [==============================] - 2s 14ms/step - loss: 1.1289 - accuracy: 0.4072 - val_loss: 1.0478 - val_accuracy: 0.4474
Epoch 4/200
140/142 [============================>.] - ETA: 0s - loss: 1

In [115]:
# Check accuracy
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.5890834331512451


In [116]:
X_test[1]

array([-4.6317276e+02,  4.4716148e+01, -1.4099199e+01,  1.4221636e+00,
       -2.2469744e+01, -4.9795675e+00, -1.3583936e+01,  1.7993051e+00,
       -1.4784613e+01, -7.4627652e+00,  4.2893818e-01, -2.8006210e+00,
       -5.3715777e-01, -5.4748554e+00,  5.1840215e+00, -4.1992879e+00,
       -1.5206343e+00, -2.8261170e+00, -3.3200803e+00, -1.5136985e+00,
        1.8531524e+00,  3.4997027e+00,  4.9135427e+00,  2.7945752e+00,
        7.9897308e-01,  4.6146360e-01,  4.0564874e-01,  1.7399962e-01,
        7.3732927e-02,  3.7308503e-02,  7.9813480e-02,  2.0730714e-01,
        8.3758041e-02,  3.0556312e-02,  2.4447298e-02,  1.0705763e-02,
        2.5716284e-02,  1.5574337e-02,  7.5092423e-04,  1.0674194e-07],
      dtype=float32)

In [117]:
model.predict(X_test)

31/31 [==============================] - 0s 3ms/step


array([[0.6877256 , 0.24707949, 0.06519488],
       [0.18302049, 0.571271  , 0.24570845],
       [0.3802635 , 0.44904304, 0.17069343],
       ...,
       [0.44623992, 0.4074413 , 0.14631878],
       [0.00139719, 0.98171633, 0.01688646],
       [0.70175713, 0.23777129, 0.06047168]], dtype=float32)

In [118]:
import numpy as np
from sklearn.metrics import classification_report

# Assuming you have already trained your ANN model and obtained the predictions and true labels
predictions = model.predict(X_test)  # Replace X_test with your test data
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)  # Replace y_test with your true labels

31/31 [==============================] - 0s 3ms/step


In [119]:
# Calculate precision, recall, and F1 score
report = classification_report(true_labels, predicted_labels)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.60      0.86      0.71       408
           1       0.56      0.56      0.56       384
           2       0.69      0.05      0.09       179

    accuracy                           0.59       971
   macro avg       0.62      0.49      0.45       971
weighted avg       0.60      0.59      0.54       971



In [ ]:
# BUILD A CNN MODEL

In [ ]:
# Preprocess the data to match proper input shape for CNN

In [ ]:
# Define the Model

In [ ]:
# Compile the model with loss and optimizer

In [ ]:
# Train the model

In [ ]:
# Check the accuracy score

In [ ]:
# BUILD A LSTM MODEL

In [ ]:
# Preprocess the data to match proper input shape for LSTM

In [ ]:
# Define the Model

In [ ]:
# Compile the model with loss and optimizer


In [ ]:
# Train the model

In [ ]:
# Check the accuracy score
